In [1]:
import os
import sys
import tqdm
import numpy as np
import scipy as scp
import pandas as pd
import seaborn as sns
import nibabel as nib
import matplotlib.pyplot as plt

from itertools import chain
from importlib import reload
from itertools import groupby
from matplotlib.lines import Line2D
from nilearn.plotting import plot_design_matrix
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix

/home/john/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
usrnm = "enter" # laptop
# usrnm = "John"  # desktop
sys.path.append(f"/mnt/c/Users/{usrnm}/OneDrive - University of Toronto/Labs/Farb Lab/Dissertation/scripts")

In [3]:
# importing custom packages

import input_import as inm
import preprocessing as pp
import roi_tseries as roits
import fmri_clustering as fmric
import sliding_time_windows as stwins

ImportError: cannot import name 'maskers' from 'nilearn' (/home/john/.local/lib/python3.8/site-packages/nilearn/__init__.py)

# Define test dataset

In [4]:
subj="2357ZL"

rest_nii=f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz"
sart_nii=f"/mnt/d/doctoral_thesis/SART/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz"

sart_behav=f"/mnt/d/doctoral_thesis/OSU_data/behav/{subj}_GoNogo.mat"

## Defining SART Events

In [5]:
subj_behav=scp.io.loadmat(sart_behav)

In [6]:
# simple contrast of Experimental (1) - Control (2)
subj_behav_pd = pd.DataFrame(data=subj_behav["rec"], 
                             columns=["iTrial","iCondition","Letters","RT","startSecs"])

In [7]:
subj_events = {"onset": [], "trial_type": [], "modulation": [], "duration": []}
for row in range(subj_behav_pd.shape[0]):
    subj_events["onset"].append(subj_behav_pd.loc[row,"startSecs"])
    subj_events["trial_type"].append("go" if subj_behav_pd.loc[row,"iCondition"]==1 else "nogo")
    subj_events["modulation"].append(-1 if subj_behav_pd.loc[row,"iCondition"]==1 else 1)
    subj_events["duration"].append(0.5)

In [8]:
subj_events=pd.DataFrame.from_dict(subj_events)

In [9]:
subj_events

,onset,trial_type,modulation,duration
0,0.0,go,-1,0.5
1,4.0,go,-1,0.5
2,8.0,go,-1,0.5
3,10.0,nogo,1,0.5
4,14.0,go,-1,0.5
...,...,...,...,...
95,324.0,go,-1,0.5
96,326.0,go,-1,0.5
97,328.0,go,-1,0.5
98,332.0,go,-1,0.5


# SART GLM

## Construct the Design Matrix

In [10]:
nib_sart = nib.load(sart_nii)

In [11]:
frametimes = np.arange(nib_sart.header.get_data_shape()[-1]) * nib_sart.header.get_zooms()[-1] # nscans * TR

In [12]:
X = make_first_level_design_matrix(frame_times=frametimes, drift_model="polynomial", 
                                   events=subj_events, 
                                   hrf_model="SPM", high_pass=None)

A 'modulation' column was found in the given events data and is used.


## Running the GLM

In [13]:
sart_glm = FirstLevelModel(mask_img=False, minimize_memory=False, standardize=True)
sart_glm = sart_glm.fit(sart_nii, design_matrices=X)

/home/john/.local/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:63: UserWarning: Mean values of 0 observed.The data have probably been centered.Scaling might not work as expected
  warn('Mean values of 0 observed.'


## Contrasts

In [14]:
contrast_matrix = np.eye(X.shape[1])
basic_contrasts = dict([(column, contrast_matrix[i])
                        for i, column in enumerate(X.columns)])

In [15]:
basic_contrasts

{'go': array([1., 0., 0., 0.]),
 'nogo': array([0., 1., 0., 0.]),
 'drift_1': array([0., 0., 1., 0.]),
 'constant': array([0., 0., 0., 1.])}

In [16]:
sart_zmap = sart_glm.compute_contrast(basic_contrasts["nogo"] - basic_contrasts["go"], 
                                      output_type="z_score")

In [17]:
sart_zmap.shape

(91, 109, 91)

## Saving Results

In [18]:
spm_dir=f"/mnt/d/doctoral_thesis/Rest/{subj}/spm"
try:
    os.mkdir(spm_dir)
except OSError as error:
    print(error)

[Errno 17] File exists: '/mnt/d/doctoral_thesis/Rest/2357ZL/spm'


In [19]:
# save imaging results
nib.save(sart_zmap, os.path.join(spm_dir, "SART_nogo-go_zmap.nii.gz"))

In [20]:
# save design data
X.to_csv(os.path.join(spm_dir, "SART_design.csv"), header=True, index=False)
np.save(os.path.join(spm_dir, "SART_params.npy"), sart_glm.get_params())

# Connectome Test

In [21]:
reload(roits)

<module 'roi_tseries' from '/mnt/c/Users/John/OneDrive - University of Toronto/Labs/Farb Lab/Dissertation/scripts/roi_tseries.py'>

In [22]:
tmp = roits.atlas_tseries(
        nii_path=f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz",
        spm=os.path.join(spm_dir, "SART_nogo-go_zmap.nii.gz"),
        atlas="/mnt/d/doctoral_thesis/atlases/brainnetome/BN_Atlas_246_2mm.nii.gz",
        atlas_data="/mnt/d/doctoral_thesis/atlases/brainnetome/subregion_func_network_simplified.csv",
        output=f"/mnt/d/doctoral_thesis/Rest/{subj}/sliding_timewindows/roi_tcourses.npy"
        )

0: roi #165
1: roi #177
2: roi #178
3: roi #211
4: roi #212
5: roi #213
6: roi #214
7: roi #215
8: roi #216
9: roi #217
10: roi #218
11: roi #219
12: roi #220
13: roi #221
14: roi #222
15: roi #223
16: roi #224
17: roi #225
18: roi #226
19: roi #227
20: roi #228
21: roi #229
22: roi #230
23: roi #231
24: roi #232
25: roi #233
26: roi #234
27: roi #235
28: roi #236
29: roi #237
30: roi #238
31: roi #239
32: roi #240
33: roi #241
34: roi #242
35: roi #243
36: roi #244
37: roi #245
38: roi #246
39: roi #105
40: roi #106
41: roi #108
42: roi #112
43: roi #113
44: roi #114
45: roi #119
46: roi #120
47: roi #135
48: roi #136
49: roi #151
50: roi #152
51: roi #182
52: roi #189
53: roi #190
54: roi #191
55: roi #192
56: roi #193
57: roi #194
58: roi #195
59: roi #196
60: roi #197
61: roi #198
62: roi #199
63: roi #200
64: roi #202
65: roi #203
66: roi #204
67: roi #205
68: roi #206
69: roi #207
70: roi #208
71: roi #209
72: roi #210
73: roi #9
74: roi #10
75: roi #53
76: roi #54
77: roi #57
78